In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4609352/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4609352/


23/08/28 09:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0767:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230828-0934-0.2.113-cf32652c5077.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [5]:
def run_pca_no_filter(mtx, mtx_path, suffix):
    
    for_pca = mtx.sample_rows(0.1)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores_no_filter = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+suffix+mtx_path)
    
    return(mtx)

### read in the polish without zeros subset the controls randomly allocated (to have 100 control samples), the matrix table already has outliers removed

In [6]:
pw_vs_gts = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_more_pw_vs_gts.mt'
)

In [7]:
pw_vs_gts.count()

(3041614, 3040)

In [8]:
pw_vs_gts.aggregate_cols(hl.agg.counter(pw_vs_gts.sex))

{'F': 450, 'M': 2550, None: 40}

In [9]:
samples_to_exclude = ['S_7288', 'S_7289', 'S_7290', 'WGS_6827', 'WGS_6835', 'WGS_5', 'WGS_8']
#these are low quality samples to exclude

pw_vs_gts = pw_vs_gts.filter_cols(
    hl.literal(samples_to_exclude).contains(pw_vs_gts.s),
    keep = False
)

#exclude duplicated sample 
pw_vs_gts = pw_vs_gts.distinct_by_col()

In [10]:
pw_vs_gts.count()

(3041614, 3037)

In [11]:
pw_vs_gts.aggregate_cols(hl.agg.counter(pw_vs_gts.sex))

{'F': 450, 'M': 2550, None: 37}

In [12]:
# I am going to divide the controls into 25 ~equal groups with 16 females and 84 males each

pw_vs_gts = pw_vs_gts.annotate_cols(
    subset_no = hl.if_else(
        pw_vs_gts.group == 'GTS',
        0,
        hl.rand_int32(1, 26, seed=0)
    )
)

In [13]:
mts = []

for i in list(range(1, 26)):
    
    to_keep_f = pw_vs_gts.filter_cols(pw_vs_gts.subset_no == i)
    
    to_keep_f = to_keep_f.filter_cols(
        to_keep_f.sex == 'F'
    )
    
    if (to_keep_f.count()[1] >= 16):
        
        to_keep_f = to_keep_f.s.take(16)
    
    else:

        to_keep_f = to_keep_f.s.collect()
    
    n_males = 100 - len(to_keep_f)
    
    to_keep_m = pw_vs_gts.filter_cols(pw_vs_gts.subset_no == i)
    
    to_keep_m = to_keep_m.filter_cols(
        to_keep_m.sex == 'M'
    )
    
    to_keep_m = to_keep_m.s.take(n_males)
    
    to_keep = to_keep_m + to_keep_f 

    
    mt = pw_vs_gts.filter_cols(
        (pw_vs_gts.subset_no == 0) | (hl.literal(to_keep).contains(pw_vs_gts.s))      
    )
    
    print(mt.count())
    print(mt.aggregate_cols(hl.agg.counter(mt.sex)))
        
    mts.append(mt)

(3041614, 137)
{'F': 14, 'M': 86, None: 37}
(3041614, 137)
{'F': 10, 'M': 90, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 15, 'M': 85, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 13, 'M': 87, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 13, 'M': 87, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 14, 'M': 86, None: 37}
(3041614, 137)
{'F': 16, 'M': 84, None: 37}
(3041614, 137)
{'F': 15, 'M': 85

In [14]:
mts_paths = []

for i in list(range(1,26)):
    mts_paths.append(str(i)+'.mt')

In [15]:
for idx, mt in enumerate(mts):
    mts[idx] = run_pca_no_filter(mt, mts_paths[idx], 'gts_vs_pol_subseted')

2023-08-27 19:04:21.663 Hail: INFO: wrote matrix table with 303736 rows and 137 columns in 489 partitions to /localfs/4609352/subset_gts_vs_pol_subseted1.mt
    Total size: 2.22 GiB
    * Rows/entries: 2.22 GiB
    * Columns: 22.95 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  1759 rows (13.06 MiB)
2023-08-27 19:04:55.140 Hail: INFO: hwe_normalize: found 279035 variants after filtering out monomorphic sites.
2023-08-27 19:04:59.152 Hail: INFO: pca: running PCA with 20 components.../ 489]
2023-08-27 19:05:36.533 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/4609352/tmp_hail/persist_table0JYl6DoR2e
    Total size: 22.62 KiB
    * Rows: 0.00 B
    * Globals: 22.62 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-08-27 19:07:22.363 Hail: INFO: wrote matrix table with 3041614 rows and 137 columns in 489 partitions to /localfs/4609352/after_pca_no_filters_gts_vs_pol_subseted1.mt
    Total size: 21.97 GiB
